# Capstone Data Science Project - with Foursquare API

## Introduction/Business Problem

Although modern vegan and vegetarian cuisine already is and becomes more popular in San Diego, CA, it seems like there is still much space for fine-dining plant-based restaurants and that some areas in San Diego County could be a good pick for embodying that idea.

The purpose of that project is to analyze chosen areas in San Diego County to find out where it could be a best choice to open upscale vegetarian/vegan restaurant.

While exploring in search for a good area to start that kind of business we consider:
+ amount of restaurants nearby
+ amount of vegetarian/vegan restaurants nearby
+ median household income in the neighborhood
+ distance from the main street/central area or other interesting attractions

## Data

Following resourses will be used to extract informations needed:

+ **Google Maps API geocoding** to find geolocations of points of interests 
+ **Foursquare API** for exploring neighborhoods, their venues, restaurants and attractions
+ **Median Household Income for San Diego County from the Census Bureau from datausa.io** website - csv including census geoid and median household income
+ **FCC Api** to convert geolocations to census geoid, to extract neighborhoods of interests from the Median Household Income csv


In [4]:
areas = ['West F Street, Encinitas, California',
'13th St, Del Mar, California',
'Girard Avenue, Village of La Jolla, California',
'Mission Blvd, Pacific Beach, California',
'University Av, Hillcrest, San Diego, California',
'Orange Ave, Coronado, California',
'North Park Way, North Park, San Diego, California',
'Rosecrans St, Point Loma, San Diego, California',
'Plaza St, Solana Beach, California',
'300 Mission Ave, Oceanside, California',
'600 Carlsbad Village Drive, Carlsbad, California']

In [24]:
import requests
import pandas as pd

In [2]:
GOOGLE_API_KEY=''

In [70]:
neighborhoods = pd.DataFrame({'col1': [], 'col2': [], 'col3': [], 'col4': [], 'col5': [], 'col6': [], 'col7': [], 'lat': [], 'lng': []})    

In [71]:
def get_coords(areas, neighborhoods):
    for area in areas:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(GOOGLE_API_KEY, area)
        response = requests.get(url).json()
        zipcode = response['results'][0]['address_components'][5]['short_name'] if len(response['results'][0]['address_components']) < 7 else response['results'][0]['address_components'][6]['short_name'] 
        neighborhoods = neighborhoods.append({
            'col1': response['results'][0]['address_components'][0]['short_name'],
            'col2': response['results'][0]['address_components'][1]['short_name'],
            'col3': response['results'][0]['address_components'][2]['short_name'],
            'col4': response['results'][0]['address_components'][3]['short_name'],
            'col5': response['results'][0]['address_components'][4]['short_name'],
            'col6': response['results'][0]['address_components'][5]['short_name'],
            'col7': zipcode,
            'lat': response['results'][0]['geometry']['location']['lat'],
            'lng': response['results'][0]['geometry']['location']['lng']
        }, ignore_index=True)
    return neighborhoods 

In [72]:
df_neighborhoods = get_coords(areas, neighborhoods)

In [74]:
df_neighborhoods.drop(['col6', 'col5'], axis=1, inplace=True)

In [93]:
df_neighborhoods.iat[9, 0] = '300 Mission Ave'
df_neighborhoods.iat[9, 1] = 'Oceanside'
df_neighborhoods.iat[10, 0] = '600 Carlsbad Village Dr'
df_neighborhoods.iat[10, 1] = 'Carlsbad'

In [95]:
df_neighborhoods.drop(['col3', 'col4'], axis=1, inplace=True)

In [99]:
df_neighborhoods.rename(columns={'col1': 'street', 'col2': 'neigborhood', 'col7': 'zipcode'}, inplace=True)

In [100]:
df_neighborhoods

,street,neigborhood,zipcode,lat,lng
0,W F St,Encinitas,92024,33.043216,-117.294944
1,13th St,Del Mar,92014,32.957393,-117.265067
2,Girard Ave,Village of La Jolla,92037,32.843179,-117.273384
3,Mission Blvd,Pacific Beach,92109,32.793680,-117.254593
4,University Ave,Hillcrest,US,32.748499,-117.154809
5,Orange Ave,Coronado,92118,32.690154,-117.177271
6,North Park Way,North Park,92104,32.747411,-117.127709
7,Rosecrans St,Point Loma,US,32.724732,-117.229103
8,Plaza St,Solana Beach,92075,32.991698,-117.272541
9,300 Mission Ave,Oceanside,US,33.195067,-117.381072
